In [1]:
import pandas as pd
import numpy as np
import nltk
import csv

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import keras
from keras import layers, optimizers, models
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
import LanguageModel as LM
import time

import csv
import random

D:\Programme\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def generate_model():
    # Initialize the model to have sequential forward propagation
    network = models.Sequential() 
    
    # Define model's architecture    
#     network.add(layers.Conv1D(
#     filters=64,
#     kernel_size= 2,
#     padding='Same',
#     strides=1,
#     activation='relu',
#     input_shape=(68,1),
#     kernel_initializer='he_normal',
#     bias_initializer='zeros'
#     ))
    
#     network.add(layers.Conv1D(
#     filters=64,
#     kernel_size=3,
#     padding='Same',
#     strides=1,
#     activation='relu',
#     kernel_initializer='he_normal',
#     bias_initializer='zeros'
#     ))
    
    network.add(layers.Conv1D(
    filters=64,
    kernel_size=5,
    padding='Same',
    strides=1,
    input_shape=(70,47),
    activation='relu',
    kernel_initializer='he_normal',
    bias_initializer='zeros'
    ))
    
    network.add(layers.Conv1D(
    filters=64,
    kernel_size=7,
    padding='Same',
    strides=1,
    activation='relu',
    kernel_initializer='he_normal',
    bias_initializer='zeros'
    ))
    
#     network.add(layers.Conv1D(
#     filters=64,
#     kernel_size=9,
#     padding='Same',
#     strides=1,
#     activation='relu',
#     kernel_initializer='he_normal',
#     bias_initializer='zeros'
#     ))
    
    network.add(layers.Flatten())
    
    network.add(layers.BatchNormalization())
    
    network.add(layers.Dropout(0.3))
    
    network.add(layers.Dense(
    3,
    activation='softmax',
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros'
    ))
    
    return network

In [3]:
#Name = './sequenceData/Model/GoodEnough'
Name = './sequenceData/Model/LanguageClassification-v1.0'


In [4]:

# learning rate and decay
learning_rate = 0.01
decay = 1e-3

# optimizer
optimizer = optimizers.Adam()

# loss
loss = "categorical_crossentropy"

# metrics
metrics = ["categorical_accuracy"]

# dropout
dropout = 0.3

# compile the model
model = generate_model()
model.compile(optimizer, loss, metrics)
model.summary()
model.load_weights(Name)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 70, 64)            15104     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 70, 64)            28736     
_________________________________________________________________
flatten_1 (Flatten)          (None, 4480)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 4480)              17920     
_________________________________________________________________
dropout_1 (Dropout)          (None, 4480)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 13443     
Total params: 75,203
Trainable params: 66,243
Non-trainable params: 8,960
_________________________________________________________________


In [34]:
DTdata = []
directoryDT = './sequenceData/Eminem_tweets.csv'
#directoryDT = './sequenceData/TestDonaldTrump_tweets.csv'
#DTdata_pd = pd.read_csv(directoryDT, delimiter='|')
with open(directoryDT, 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='|')
    for row in spamreader:
        if row !=[]:
            r=[]
            for col in row:
                r.append(float(col))      
            DTdata.append(np.array(r))
    csvfile.closed
    
DTdata_np = np.array(DTdata)
x_data_DT = np.zeros((len(DTdata_np), 70))
x_data_DT_oneHot = []

for i in range(len(DTdata_np)):   
    newOneHot = []
    x_data_DT[i, :len(DTdata_np[i])] = DTdata_np[i]
    for e in x_data_DT[i]:
        newOneHot.append(to_categorical(e, num_classes=47))
    x_data_DT_oneHot.append(np.array(newOneHot))

x_data_DT_oneHot = np.array(x_data_DT_oneHot)
test_data = x_data_DT_oneHot.reshape(-1, 70, 47)


x_data_DT_aug = np.zeros((len(DTdata_np), 70))
x_data_DT_oneHot_aug = []
for i in range(len(DTdata_np)): 
    newOneHot = []
    row = np.array(DTdata_np[i], copy=True)  
    np.random.shuffle(row) 
    x_data_DT_aug[i, :len(row)] = row
    for e in x_data_DT_aug[i]:
        newOneHot.append(to_categorical(e, num_classes=47))
    x_data_DT_oneHot_aug.append(np.array(newOneHot))
    
x_data_DT_oneHot_aug = np.array(x_data_DT_oneHot_aug)
test_data_rd = x_data_DT_oneHot_aug.reshape(-1, 70, 47)
    
    
    

In [35]:
predictions = model.predict(test_data)
predictionsRD = model.predict(test_data_rd)

In [36]:
u, c = np.unique(np.argmax(predictions, axis=1), return_counts = True)
urd, crd = np.unique(np.argmax(predictionsRD, axis=1), return_counts = True)

In [37]:
emineml = sum(c)
print("eminem ",u, c/emineml)
print("random ",urd, crd/emineml)


eminem  [0 1 2] [0.14071038 0.79371585 0.06557377]
random  [0 1 2] [0.00546448 0.05874317 0.93579235]


In [38]:

percantages = [0.75, -0.25, 0]

b = predictions *percantages
d = np.sum(predictions * percantages, axis= 1)
e = np.sum(predictionsRD * percantages, axis= 1)

d[:] += 0.25
d[:] = d[:]/1
e[:] += 0.25
e[:] = e[:]/1

print(np.mean(d), np.mean(e))

0.24559732721834987 0.24417738624862675


In [39]:
eminem = [0.24559732721834987, 0.24417738624862675]
dt = [0.608606853798298, 0.24736444697725454]